# 건축 도면 데이터셋 MinIO 업로드 스크립트

Training과 Validation 데이터의 ZIP 파일들을 MinIO 서버에 업로드합니다.

In [ ]:
import os  
from pathlib import Path  
import sys  
import zipfile  
import tempfile  
import shutil  
from tqdm.notebook import tqdm  

# src 디렉토리를 파이썬 경로에 추가  
project_root = Path.cwd().parent  
sys.path.append(str(project_root))  

from data_engineering.obj_storage.boto3 import MinIOClient  
from data_engineering.update_objectfile import upload_file_with_metadata

In [15]:
# MinIO 클라이언트 초기화
minio_client = MinIOClient()

# 데이터셋 기본 경로 설정
#DATASET_PATH = Path("../../temp/239.건축 도면 데이터/01-1.정식개방데이터")
DATASET_PATH = project_root.parent / "temp" / "239.건축 도면 데이터" / "01-1.정식개방데이터"

# 처리할 디렉토리 구조 정의
DIRECTORIES = [
    "Training/01.원천데이터",
    "Training/02.라벨링데이터",
    "Validation/01.원천데이터",
    "Validation/02.라벨링데이터"
]

# 버킷 이름 설정
BUCKET_NAME = "tada"

# 경로가 올바른지 확인
print(f"데이터셋 절대 경로: {DATASET_PATH.absolute()}")
print(f"경로 존재 여부: {DATASET_PATH.exists()}")

데이터셋 절대 경로: g:\Dropbox\OssmMath\52_AI-Project\TADA\temp\239.건축 도면 데이터\01-1.정식개방데이터
경로 존재 여부: True


In [19]:
print(f"데이터셋 경로: {DATASET_PATH}")  
print(f"경로 존재?: {DATASET_PATH.exists()}")  

# 디렉토리 구조 확인  
for dir_path in DIRECTORIES:  
    full_path = DATASET_PATH / dir_path  
    print(f"\n{dir_path}:")  
    print(f"경로: {full_path}")  
    print(f"존재?: {full_path.exists()}")  
    if full_path.exists():  
        print("포함된 파일들:")  
        for file in full_path.iterdir():  
            print(f"  - {file.name}")  

데이터셋 경로: g:\Dropbox\OssmMath\52_AI-Project\TADA\temp\239.건축 도면 데이터\01-1.정식개방데이터
경로 존재?: True

Training/01.원천데이터:
경로: g:\Dropbox\OssmMath\52_AI-Project\TADA\temp\239.건축 도면 데이터\01-1.정식개방데이터\Training\01.원천데이터
존재?: True
포함된 파일들:
  - TS_OBJ_1.zip
  - TS_OBJ_2.zip
  - TS_OCR_1.zip
  - TS_OCR_2.zip
  - TS_SPA_1.zip
  - TS_SPA_2.zip
  - TS_STR_1.zip
  - TS_STR_2.zip

Training/02.라벨링데이터:
경로: g:\Dropbox\OssmMath\52_AI-Project\TADA\temp\239.건축 도면 데이터\01-1.정식개방데이터\Training\02.라벨링데이터
존재?: True
포함된 파일들:
  - TL_OBJ.zip
  - TL_OCR.zip
  - TL_SPA.zip
  - TL_STR.zip

Validation/01.원천데이터:
경로: g:\Dropbox\OssmMath\52_AI-Project\TADA\temp\239.건축 도면 데이터\01-1.정식개방데이터\Validation\01.원천데이터
존재?: True
포함된 파일들:
  - VS_OBJ.zip
  - VS_OCR.zip
  - VS_SPA.zip
  - VS_STR.zip

Validation/02.라벨링데이터:
경로: g:\Dropbox\OssmMath\52_AI-Project\TADA\temp\239.건축 도면 데이터\01-1.정식개방데이터\Validation\02.라벨링데이터
존재?: True
포함된 파일들:
  - VL_OBJ.zip
  - VL_OCR.zip
  - VL_SPA.zip
  - VL_STR.zip


In [21]:
# 버킷이 없으면 생성
if not minio_client.check_bucket_exists(BUCKET_NAME):
    minio_client.create_bucket(BUCKET_NAME)
    print(f"버킷 '{BUCKET_NAME}' 생성 완료")
else:
    print(f"버킷 '{BUCKET_NAME}' 이미 존재함")


# DB 테이블 생성  
from data_engineering.db.init_db import create_object_storage_table  
print("DB 테이블 생성 시도...")  
create_object_storage_table()  

버킷 'tada' 이미 존재함
DB 테이블 생성 시도...
object_storage 테이블이 성공적으로 생성되었습니다.


True

In [22]:
# 지원하는 이미지 확장자
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".bmp"}

def is_image_file(filename):
    return any(filename.lower().endswith(ext) for ext in IMAGE_EXTENSIONS)

# 임시 디렉토리 생성
temp_dir = Path(tempfile.mkdtemp())
print(f"임시 디렉토리 생성됨: {temp_dir}")

# ZIP 파일 처리 부분 수정  
try:  
    from tqdm.notebook import tqdm  
    
    # 각 디렉토리 처리  
    for dir_path in DIRECTORIES:  
        current_path = DATASET_PATH / dir_path  
        print(f"\n처리 중인 디렉토리: {dir_path}")  
        
        # ZIP 파일 목록 가져오기  
        zip_files = list(current_path.glob("*.zip"))  
        print(f"발견된 ZIP 파일 수: {len(zip_files)}")  
        
        # 각 ZIP 파일 처리  
        for zip_path in tqdm(zip_files, desc="ZIP 파일 처리 중"):  
            # ZIP 파일 이름을 카테고리로 사용하되, 경로 구조도 보존  
            category = f"{dir_path}/{zip_path.stem}"  
            print(f"\n처리 중인 카테고리: {category}")  
            
            # ZIP 파일 압축 해제  
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:  
                # 이미지 파일만 추출  
                for file_info in tqdm(zip_ref.filelist, desc="파일 추출 중"):  
                    if is_image_file(file_info.filename):  
                        # 파일 경로에서 디렉토리 구조 유지  
                        relative_path = Path(file_info.filename)  
                        object_key = f"{category}/{relative_path}"  
                        
                        # 임시 파일로 추출  
                        temp_path = temp_dir / relative_path.name  
                        with zip_ref.open(file_info) as source, open(temp_path, 'wb') as target:  
                            shutil.copyfileobj(source, target)  
                        
                        try:  
                            # MinIO 업로드 및 메타데이터 저장  
                            success = upload_file_with_metadata(  
                                local_path=str(temp_path),  
                                category=category,  
                                bucket=BUCKET_NAME,  
                                object_path=object_key  
                            )  
                            if not success:  
                                print(f"업로드 또는 메타데이터 저장 실패: {object_key}")  
                        except Exception as e:  
                            print(f"처리 실패: {object_key} - {str(e)}")  
                        
                        # 임시 파일 삭제  
                        temp_path.unlink()  

finally:  
    # 임시 디렉토리 정리  
    shutil.rmtree(temp_dir)  
    print("임시 디렉토리 정리 완료")

임시 디렉토리 생성됨: C:\Users\dongr\AppData\Local\Temp\tmp16qw5ekz

처리 중인 디렉토리: Training/01.원천데이터
발견된 ZIP 파일 수: 8


ZIP 파일 처리 중:   0%|          | 0/8 [00:00<?, ?it/s]


처리 중인 카테고리: Training/01.원천데이터/TS_OBJ_1


파일 추출 중:   0%|          | 0/6058 [00:00<?, ?it/s]


처리 중인 카테고리: Training/01.원천데이터/TS_OBJ_2


파일 추출 중:   0%|          | 0/2041 [00:00<?, ?it/s]


처리 중인 카테고리: Training/01.원천데이터/TS_OCR_1


파일 추출 중:   0%|          | 0/6057 [00:00<?, ?it/s]

Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada/Training/01.%EC%9B%90%EC%B2%9C%EB%8D%B0%EC%9D%B4%ED%84%B0/TS_OCR_1/%5CAPT_FP_OCR_733807493.PNG?uploadId=ZDY0N2RlZjMtYWRlMi00MWY1LThkMWUtZmQ5YjA5NTE5ZTE4LmFiOTdjYzJjLTU0ZjYtNDliMC05YzYwLWFlOTRjYjc5MTQzOHgxNzMyNDg2MzU3NDM5OTg4MzAz&partNumber=2".
업로드 또는 메타데이터 저장 실패: Training/01.원천데이터/TS_OCR_1/\APT_FP_OCR_733807493.PNG
Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada".
업로드 또는 메타데이터 저장 실패: Training/01.원천데이터/TS_OCR_1/\APT_FP_OCR_731029197.PNG
Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada".
업로드 또는 메타데이터 저장 실패: Training/01.원천데이터/TS_OCR_1/\APT_FP_OCR_729581217.PNG
Upload error: Connection was closed before we received a valid response from endpoint URL: "http://175.214.62.133:50008/tada".
업로드 또는 메타데이터 저장 실패: Training/01

KeyboardInterrupt: 

In [ ]:
# 업로드된 파일과 메타데이터 확인  
from Extractor.loader.db.connector import Database  
import os  
from dotenv import load_dotenv  

# MinIO 확인  
uploaded_files = minio_client.list_objects(BUCKET_NAME)  
print(f"MinIO 업로드된 총 파일 수: {len(uploaded_files)}")  

# DB 확인  
load_dotenv(Path(__file__).parent.parent / '.env')  
db = Database(  
    host=os.getenv('DB_HOST'),  
    port=int(os.getenv('DB_PORT')),  
    user=os.getenv('DB_USER'),  
    password=os.getenv('DB_PASSWORD'),  
    database=os.getenv('DB_NAME')  
)  

query = "SELECT COUNT(*) as count FROM object_storage"  
result = db.getter(query)  
print(f"\nDB에 저장된 메타데이터 수: {result[0]['count']}")  

# 처음 몇 개의 레코드 확인  
query = "SELECT * FROM object_storage LIMIT 5"  
records = db.getter(query)  
print("\nDB 레코드 샘플 (처음 5개):")  
for record in records:  
    print(f"- {record['category']}: {record['file_name']}")